# ⌚️ Inferring Relationships in Wearable-Sourced Health Data

Ths notebook explores, models and attempts to explain latent patterns in data sourced from my Apple Watch. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

## Data

Define an extraction step to ingest the raw data, which we'll skil if the post-processed data is available to avoid oversharing PII.

### Extraction 

Apple stores health data in a proprietary XML format which can be exported in bulk. 

Note: Parsing the HealthKit format below is done with the 

We'll limit the temporal analysis required here by summing within a calendar day. This will yield some reasonable and intuitve metrics we can try to predict off of. The consolidation of the relevant tables is surprisingly straightforward once we apply this constraint: 

HKCategoryTypeIdentifierSleepAnalysis
- on end_date got range hours of sleep 

HKQuantityTypeIdentifierActiveEnergyBurned
- on end_date burned value calories
- sum value by date

HKQuantityTypeIdentifierAppleExerciseTime
- on end_date exercised for 1 min
- sum value by end_date

HKQuantityTypeIdentifierAppleStandTime
- on end_date stood for value mins
- sum value on end_date

HKQuantityTypeIdentifierBasalEnergyBurned
- on end_date calories burned on end_date
- sum value on end_date

HKQuantityTypeIdentifierFlightsClimbed
- on end_date 1 flight climbed 
- sum value on end_date

HKQuantityTypeIdentifierHeartRate
- instantaneous HR measurements at end_date
- perhaps take max value of columnd within date

HKQuantityTypeIdentifierOxygenSaturation
- instantaneous on end date
- take max and min for the day

HKQuantityTypeIdentifierPhysicalEffort
- instanteous measure of energy on end_date
- take max and min for the day 

HKQuantityTypeIdentifierRespiratoryRate
- instantaneous 
- take average? 

HKQuantityTypeIdentifierRestingHeartRate
- value has an average over a short period (length of end_date - start_date)
- take min and max for the day 

HKQuantityTypeIdentifierStepCount
- value for short window 
- sum for end_date

HKQuantityTypeIdentifierTimeInDaylight
- sum value for end_date

In [2]:
INCLUDE_FLAGS = (
    "HKCategoryTypeIdentifierSleepAnalysis",
    "HKQuantityTypeIdentifierActiveEnergyBurned",
    "HKQuantityTypeIdentifierAppleExerciseTime",
    "HKQuantityTypeIdentifierAppleStandTime",
    "HKQuantityTypeIdentifierBasalEnergyBurned",
    "HKQuantityTypeIdentifierFlightsClimbed",
    "HKQuantityTypeIdentifierHeartRate",
    "HKQuantityTypeIdentifierOxygenSaturation",
    "HKQuantityTypeIdentifierPhysicalEffort",
    "HKQuantityTypeIdentifierRespiratoryRate",
    "HKQuantityTypeIdentifierRestingHeartRate",
    "HKQuantityTypeIdentifierStepCount",
    "HKQuantityTypeIdentifierTimeInDaylight",
)

❗️ The library we use for parsing HealthKit data reliese on some enum behavior that wasn't stabilized in python 3.11, however Colab is still running on 3.11 so we can't upgrade. Below GPT-5-supplied workaround to patch enum behavior is necessary if a full parsing operation is run (shoudn't be necessary if running on colab though)

In [3]:
import sys, enum

if (3, 11) <= sys.version_info < (3, 12):
    _old_contains = enum.EnumMeta.__contains__
    def _patched_contains(cls, member):
        # Try the old behavior; if 3.11 raises TypeError for str, emulate 3.12:
        try:
            return _old_contains(cls, member)
        except TypeError:
            try:
                cls(member)  # attempt coercion to Enum
                return True
            except Exception:
                return False
    enum.EnumMeta.__contains__ = _patched_contains


Abstract the XML-based format and emit a dict of dataframes for each of our target tables. 

In [4]:
from apple_health_parser.utils.parser import Parser

def parse_healthkit_data(path, output_dir="./data/out", flags=INCLUDE_FLAGS): 
    """
    Go through the raw data with the help of the above import. 

    NOTE: HealthKit parsing logic courtesy of GPT-5, see https://chatgpt.com/share/68d811d3-6e00-8013-bca7-e39aa6e6f106
    """
    parser = Parser(export_file=path, output_dir=output_dir, overwrite=True)

    flags = [f for f in parser.flags if f.startswith(flags)]

    dfs = {}      # dict[str, pd.DataFrame]  -> all parsed flags you can model from
    errors = {}   # keep going even if a flag fails

    for f in flags:
        try:
            parsed = parser.get_flag_records(flag=f)   # returns ParsedData
            if not parsed.records.empty:
                dfs[f] = parsed.records
        except Exception as e:
            errors[f] = repr(e)

    print(len(dfs), list(dfs)[:8], len(errors))

    return dfs

In [5]:
def aggregate_by_date(df, date_col, val_col, op="sum", name=None):
    """
    Aggregate dataframe column by date, summing

    NOTE:  Date aggregation syntax courtesy of gpt-5, see https://chatgpt.com/share/68d82165-543c-8013-a1c4-4d5ac9d68410
    """
    groups = df.groupby(df[date_col].dt.date)
        
    agg_df = None
    match(op):         
        case "min" : 
            agg_df = groups[val_col].min().reset_index()
        case "max" : 
            agg_df = groups[val_col].max().reset_index()
        case "mean" : 
            agg_df = groups[val_col].mean().reset_index()
        case "sum" | None | _ : 
            agg_df = groups[val_col].sum().reset_index()

    kwargs = { 'columns': [name] } if name else {}
    return pd.DataFrame(agg_df[val_col].values, index=agg_df[date_col].values, **kwargs)

In [6]:
def process_healthkit_data(table_dfs):
    """
    Given a dict of dataframes with our target data container, filter and construct a unifed dataframe 
    that aggregates data by date. Note the tight coupling with parse flags above. 
    """

    agg_dfs = {}
    agg_dfs['sleep'] = aggregate_by_date(table_dfs['HKCategoryTypeIdentifierSleepAnalysis'], date_col="end_date", val_col="range", op="sum", name="sleep_h") 
    agg_dfs['energy'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierActiveEnergyBurned'], date_col="end_date", val_col="value", op="sum", name="active_cal") 
    agg_dfs['exercise'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierAppleExerciseTime'], date_col="end_date", val_col="value", op="sum", name="exercise_h") 
    agg_dfs['stand'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierAppleStandTime'], date_col="end_date", val_col="value", op="sum", name="stand_h") 
    agg_dfs['basal'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierBasalEnergyBurned'], date_col="end_date", val_col="value", op="sum", name="basal_cal") 
    agg_dfs['flights'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierFlightsClimbed'], date_col="end_date", val_col="value", op="sum", name="flights") 
    agg_dfs['hr'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierHeartRate'], date_col="end_date", val_col="value", op="max", name="max_hr") 
    agg_dfs['o2'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierOxygenSaturation'], date_col="end_date", val_col="value", op="min", name="min_o2") 
    agg_dfs['effort'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierPhysicalEffort'], date_col="end_date", val_col="value", op="sum", name="activity_mets") 
    agg_dfs['rr'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierRespiratoryRate'], date_col="end_date", val_col="value", op="mean", name="avg_rr") 
    agg_dfs['resting'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierRestingHeartRate'], date_col="end_date", val_col="value", op="mean", name="avg_resting_hr") 
    agg_dfs['step'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierStepCount'], date_col="end_date", val_col="value", op="sum", name="steps") 
    agg_dfs['daylight'] = aggregate_by_date(table_dfs['HKQuantityTypeIdentifierTimeInDaylight'], date_col="end_date", val_col="value", op="sum", name="daylight_h") 

    # Migrat to float value for sleep duration from timedelta.  NOTE: GPT-5-supplied conversion syntax
    agg_dfs['sleep'].sleep_h = agg_dfs['sleep'].sleep_h / np.timedelta64(1, "h")

    # Join our table DFs
    df = pd.DataFrame()
    df = df.join(list(agg_dfs.values()), how="outer")
    #macro_df.reset_index(names='date', inplace=True)

    # Coax the string value for the date back to a numeric and sort. NOTE: type conversion courtesy of GPT-5
    df.index = pd.to_datetime(df.index, errors="coerce")
    df.sort_index(inplace=True)

    return df

If we can restore the processed data, do so, otherwise mine our data out of the full HealthKit export.

In [8]:
raw_dataset = "./data/export.zip"
processed_dataset = "./data/health.csv"

df = pd.read_csv(processed_dataset)
if df is None: 
    
    # Mine the raw export 
    table_dfs = parse_healthkit_data(raw_dataset)
    df = process_healthkit_data(table_dfs)

    # Inspect for spsarsity
    sns.heatmap(df.isna())
    
    # Avoid doing this again on subsequent runs!
    df.to_csv("dataset_path")
    